In [83]:
import pandas as pd
import numpy as np

In [84]:
dataset = pd.read_csv("D:/Projects/Zomato Delivery Dataset/Zomato Dataset.csv", sep=",")

In [85]:
dataset

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45579,0x7c09,JAPRES04DEL01,30.0,4.8,26.902328,75.794257,26.912328,75.804257,24-03-2022,11:35,11:45,Windy,High,1,Meal,motorcycle,0.0,No,Metropolitian,32
45580,0xd641,AGRRES16DEL01,21.0,4.6,0.000000,0.000000,0.070000,0.070000,16-02-2022,19:55,20:10,Windy,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian,36
45581,0x4f8d,CHENRES08DEL03,30.0,4.9,13.022394,80.242439,13.052394,80.272439,11-03-2022,23:50,24:05:00,Cloudy,Low,1,Drinks,scooter,0.0,No,Metropolitian,16
45582,0x5eee,COIMBRES11DEL01,20.0,4.7,11.001753,76.986241,11.041753,77.026241,07-03-2022,13:35,13:40,Cloudy,High,0,Snack,motorcycle,1.0,No,Metropolitian,26


In [86]:
"""
This dataset can be broken down into 3 segments
1. Delivery Person Info - Delivery_person_id, Delivery_person_age, Delivery_person_ratings
2. Conditions affecting the deliveries - Weather, Road Traffic, Vehicle Condition, Type of vehicle, multiple deliveries, Festival, City
3. Order and Delivery Info - Order date, order time, order pickup time, type of order, time taken
"""

'\nThis dataset can be broken down into 3 segments\n1. Delivery Person Info - Delivery_person_id, Delivery_person_age, Delivery_person_ratings\n2. Conditions affecting the deliveries - Weather, Road Traffic, Vehicle Condition, Type of vehicle, multiple deliveries, Festival, City\n3. Order and Delivery Info - Order date, order time, order pickup time, type of order, time taken\n'

# Cleaning Data

In [87]:
# Removing Restaurant and Delivery Location Lat, Long because we cant perform analysis on them
clean_data = dataset.drop(['Delivery_location_longitude', 'Delivery_location_latitude', 'Restaurant_latitude', 'Restaurant_longitude'], axis=1)

In [88]:
# Merging Order_date with Time_Ordered and Order_date with Time_order_picked
clean_data = clean_data.assign(
    placed_at = clean_data['Order_Date'] + ' ' + clean_data['Time_Orderd'],
    picked_at = clean_data['Order_Date'] + ' ' + clean_data['Time_Order_picked']
)

In [89]:
# Changing the dtype for placed_at and picked_at to datetime64 for analysis
clean_data = clean_data.assign(
    placed_at = pd.to_datetime(clean_data['placed_at'], errors='coerce', format='%d-%m-%Y %H:%M'),
    picked_at = pd.to_datetime(clean_data['picked_at'], errors='coerce', format='%d-%m-%Y %H:%M'
))

In [90]:
# removing the outdated columns
clean_data.drop(['Order_Date', 'Time_Order_picked', 'Time_Orderd'], axis=1, inplace=True)

In [91]:
# Reordering the columns
clean_data = clean_data[['ID', 'Delivery_person_ID', 'Delivery_person_Age', 'Delivery_person_Ratings',
                         'placed_at', 'picked_at', 'Time_taken (min)', 'Type_of_order',
                         'Weather_conditions', 'Road_traffic_density', 'Festival', 'City' ,'Vehicle_condition', 'Type_of_vehicle', 'multiple_deliveries']]

In [92]:
# handling missing values
clean_data.isna().mean() * 100

ID                          0.000000
Delivery_person_ID          0.000000
Delivery_person_Age         4.067217
Delivery_person_Ratings     4.185679
placed_at                  12.721569
picked_at                  10.984117
Time_taken (min)            0.000000
Type_of_order               0.000000
Weather_conditions          1.351351
Road_traffic_density        1.318445
Festival                    0.500176
City                        2.632503
Vehicle_condition           0.000000
Type_of_vehicle             0.000000
multiple_deliveries         2.178396
dtype: float64

In [93]:
# Since most of the data in placed_at and picked_at column was invalid that was converted into NAN, 
# so we will be filtering those records out(considering on the placed_at and picked_at column)

In [94]:
clean_data = clean_data.loc[~(clean_data['placed_at'].isna() | clean_data['picked_at'].isna())]

In [95]:
clean_data.isna().mean() * 100

ID                         0.000000
Delivery_person_ID         0.000000
Delivery_person_Age        0.488604
Delivery_person_Ratings    0.601576
placed_at                  0.000000
picked_at                  0.000000
Time_taken (min)           0.000000
Type_of_order              0.000000
Weather_conditions         0.000000
Road_traffic_density       0.000000
Festival                   0.494253
City                       2.598356
Vehicle_condition          0.000000
Type_of_vehicle            0.000000
multiple_deliveries        2.154941
dtype: float64

In [96]:
# For the multiple_deliveries column,
# there are about 2.15% of null values
# We must find out the best replacement value, to fill the null values

# So below is the distribution of Deliveries for each type of multiple delivery
clean_data['multiple_deliveries'].value_counts(normalize=True, dropna=False) * 100


# Let's Understand what each type of multiple delivery means
# 0 - No Multiple Deliveries i.e. Delivery Person only delivered to 1 customer
# 1 - One Multiple Delivery i.e. Delivery person only delivered to 2 customers
# 2 - Two Multiple Delivery i.e. Delivery person only delivered to 3 customers
# 3 - Three Multiple Delivery i.e. Delivery person only delivered to 4 customers

multiple_deliveries
1.0    61.643178
0.0    30.974101
2.0     4.436976
NaN     2.154941
3.0     0.790804
Name: proportion, dtype: float64

In [97]:
# Now For proper imputation, we must figure out the 2.15% of missing data belongs to what type of multiple deliveries
# for that we understood what is the avg time taken for the delivery for each type of multiple deliveries and 
# the avg time taken for the deliveries with missing data which comes out to be 22.56% that belongs to the category of 0-type of multiple Deliveries
# Therefore for imputation, 0 is the appropriate fill value.
clean_data.groupby(['multiple_deliveries']).agg(avg_time = ('Time_taken (min)', 'mean'), max_time= ('Time_taken (min)', 'max'), min_time=('Time_taken (min)', 'min'))

,avg_time,max_time,min_time
multiple_deliveries,,,
0.0,22.907997,54,10
1.0,26.917804,54,10
2.0,40.366645,54,31
3.0,47.867857,54,42


In [98]:
clean_data.loc[clean_data['multiple_deliveries'].isna()]['Time_taken (min)'].mean()

np.float64(22.567496723460025)

In [99]:
# After imputation
clean_data.fillna({'multiple_deliveries': 0.0}, inplace=True)

In [100]:
clean_data.groupby(['multiple_deliveries']).agg(avg_time = ('Time_taken (min)', 'mean'), max_time= ('Time_taken (min)', 'max'), min_time=('Time_taken (min)', 'min'))

,avg_time,max_time,min_time
multiple_deliveries,,,
0.0,22.885848,54,10
1.0,26.917804,54,10
2.0,40.366645,54,31
3.0,47.867857,54,42


In [101]:
clean_data.isna().mean() * 100

ID                         0.000000
Delivery_person_ID         0.000000
Delivery_person_Age        0.488604
Delivery_person_Ratings    0.601576
placed_at                  0.000000
picked_at                  0.000000
Time_taken (min)           0.000000
Type_of_order              0.000000
Weather_conditions         0.000000
Road_traffic_density       0.000000
Festival                   0.494253
City                       2.598356
Vehicle_condition          0.000000
Type_of_vehicle            0.000000
multiple_deliveries        0.000000
dtype: float64

In [102]:
clean_data['City'].value_counts(normalize=True, dropna=False) * 100

City
Metropolitian    74.776174
Urban            22.241365
NaN               2.598356
Semi-Urban        0.384105
Name: proportion, dtype: float64

In [103]:
clean_data.groupby(['City'])['Time_taken (min)'].mean()

City
Metropolitian    27.381478
Semi-Urban       49.661765
Urban            22.937016
Name: Time_taken (min), dtype: float64

In [104]:
clean_data.loc[clean_data['City'].isna()]['Time_taken (min)'].mean()

np.float64(22.220652173913045)

In [105]:
clean_data.loc[clean_data['City'].isna()]['Road_traffic_density'].value_counts(normalize=True, dropna=False) * 100

Road_traffic_density
Low       43.478261
Jam       24.565217
Medium    23.586957
High       8.369565
Name: proportion, dtype: float64

In [106]:
clean_data.loc[clean_data['City'] == "Urban"]['Road_traffic_density'].value_counts(normalize=True, dropna=False) * 100

Road_traffic_density
Low       40.419048
Jam       26.336508
Medium    23.974603
High       9.269841
Name: proportion, dtype: float64

In [107]:
# Now for the imputation of City Column,
# we have to understand what replacement value fits the best
# From the statistics above there is about 2.59% of missing data in city column
# There are 3 distinct values for the city column i.e. Metropolitan, Urban, Semi-Urban
# Lets Understand the avg delivery time for each type of city option, comparing it with the avg delivery time when city is null
# For this scenario, the avg delivery time for Urban City matches closely with the avg delivery time when the city is null,
# this suggests that it takes almost the same amount of avg time to deliver food in urban areas as compared to city with null values
# Upon Further inspection regarding the Road traffic density for Urban Area deliveries and Null city Area deliveries, The comparison is almost same

# Therefore for imputation the appropriate fill value can be Urban in the city column
clean_data.fillna({'City': 'Urban'}, inplace=True)

In [108]:
clean_data['City'].value_counts(normalize=True, dropna=False) * 100

City
Metropolitian    74.776174
Urban            24.839721
Semi-Urban        0.384105
Name: proportion, dtype: float64

In [109]:
clean_data.groupby('City')['Time_taken (min)'].mean()

City
Metropolitian    27.381478
Semi-Urban       49.661765
Urban            22.862081
Name: Time_taken (min), dtype: float64

In [110]:
clean_data.isna().mean() * 100

ID                         0.000000
Delivery_person_ID         0.000000
Delivery_person_Age        0.488604
Delivery_person_Ratings    0.601576
placed_at                  0.000000
picked_at                  0.000000
Time_taken (min)           0.000000
Type_of_order              0.000000
Weather_conditions         0.000000
Road_traffic_density       0.000000
Festival                   0.494253
City                       0.000000
Vehicle_condition          0.000000
Type_of_vehicle            0.000000
multiple_deliveries        0.000000
dtype: float64

In [111]:
clean_data['Festival'].value_counts(normalize=True, dropna=False) * 100

Festival
No     97.537210
Yes     1.968537
NaN     0.494253
Name: proportion, dtype: float64

In [112]:
clean_data.groupby('Festival')['Time_taken (min)'].mean()

Festival
No     26.034168
Yes    45.530846
Name: Time_taken (min), dtype: float64

In [113]:
clean_data.loc[clean_data['Festival'].isna()]['Time_taken (min)'].mean()

np.float64(11.16)

In [114]:
# Notice that the avg delivery time for the missing data in festival column does not match with avg delivery time for when there was and was not a festival
# also In the festival Column, about 0.49% of data is missing, which is comparatively less so instead of imputation we can perform deletion (Filtering)

In [115]:
clean_data = clean_data.loc[~clean_data['Festival'].isna()]

In [116]:
clean_data.Festival.value_counts(normalize=True, dropna=False) * 100

Festival
No     98.021685
Yes     1.978315
Name: proportion, dtype: float64

In [117]:
clean_data.isna().mean() * 100

ID                         0.000000
Delivery_person_ID         0.000000
Delivery_person_Age        0.488193
Delivery_person_Ratings    0.598887
placed_at                  0.000000
picked_at                  0.000000
Time_taken (min)           0.000000
Type_of_order              0.000000
Weather_conditions         0.000000
Road_traffic_density       0.000000
Festival                   0.000000
City                       0.000000
Vehicle_condition          0.000000
Type_of_vehicle            0.000000
multiple_deliveries        0.000000
dtype: float64

In [118]:
# Now for Age and Rating column, since these columns contains Numerical Data so performing imputation
# is not about selecting an option as the fill value that fits appropriately
# But on the basis of data distribution, a random value or synthetic data will be chosen as the replacement value
# such that it does not disturb the original distribution of data

In [119]:
clean_data['Delivery_person_Ratings'].describe()
# As you can see about 75% of deliveries are done by person with ratings between 4.5 and 5
synthetic_rating = np.random.randint(45, 51)/10

In [120]:
clean_data['Delivery_person_Age'].describe()
# since the data distribution is this case is almost symmetric
# Median as the fill value can be a good and simple option

count    35060.000000
mean        29.602852
std          5.767222
min         20.000000
25%         25.000000
50%         30.000000
75%         35.000000
max         39.000000
Name: Delivery_person_Age, dtype: float64

In [121]:
clean_data.fillna({'Delivery_person_Age': clean_data['Delivery_person_Age'].median(),
                   'Delivery_person_Ratings': synthetic_rating}, inplace=True)

In [122]:
clean_data.describe()

,Delivery_person_Age,Delivery_person_Ratings,placed_at,picked_at,Time_taken (min),Vehicle_condition,multiple_deliveries
count,35232.000000,35232.000000,35232,35232,35232.000000,35232.000000,35232.000000
mean,29.604791,4.636722,2022-03-14 10:35:59.800749312,2022-03-14 10:45:56.428814592,26.419874,0.998524,0.731012
min,20.000000,2.500000,2022-02-11 08:10:00,2022-02-11 08:15:00,10.000000,0.000000,0.000000
25%,25.000000,4.500000,2022-03-04 23:15:00,2022-03-04 23:25:00,19.000000,0.000000,0.000000
50%,30.000000,4.700000,2022-03-15 17:10:00,2022-03-15 17:22:30,26.000000,1.000000,1.000000
75%,35.000000,4.900000,2022-03-27 17:55:00,2022-03-27 18:05:00,33.000000,2.000000,1.000000
max,39.000000,5.000000,2022-04-06 23:50:00,2022-04-06 23:55:00,54.000000,2.000000,3.000000
std,5.753193,0.314267,NaN,NaN,9.356421,0.817445,0.577501


In [123]:
# Changing the dtype of columns
clean_data = clean_data.astype({'Delivery_person_Age': 'int32'})

In [124]:
clean_data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
Index: 35232 entries, 0 to 45583
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ID                       35232 non-null  object        
 1   Delivery_person_ID       35232 non-null  object        
 2   Delivery_person_Age      35232 non-null  int32         
 3   Delivery_person_Ratings  35232 non-null  float64       
 4   placed_at                35232 non-null  datetime64[ns]
 5   picked_at                35232 non-null  datetime64[ns]
 6   Time_taken (min)         35232 non-null  int64         
 7   Type_of_order            35232 non-null  object        
 8   Weather_conditions       35232 non-null  object        
 9   Road_traffic_density     35232 non-null  object        
 10  Festival                 35232 non-null  object        
 11  City                     35232 non-null  object        
 12  Vehicle_condition        35232 non-nu

In [125]:
# since this data was obtained from kaggle, so it has some inconsistencies
# for Delivery person age and ratings,
# which naturally should be fixed for each individual but according to the data,
# each delivery person has different age and rating for each delivery it makes.

# To Fix this, we are going to aggregate the age and rating (Average) for each delivery person
# and join it with the original clean data and keep the new aggregated columns, removing the old ones
aggregated_data = clean_data.groupby('Delivery_person_ID').agg(
    PersonAge = ('Delivery_person_Age', 'mean'),
    PersonRating = ('Delivery_person_Ratings', 'min')
)

In [126]:
final_data = aggregated_data.assign(
    PersonAge = aggregated_data['PersonAge'].round().astype('int32'),
    PersonRating = aggregated_data['PersonRating'].round(1)
).reset_index()

In [127]:
final_data['PersonRating'].describe()

count    1320.000000
mean        3.849318
std         0.444731
min         2.500000
25%         3.600000
50%         3.900000
75%         4.100000
max         4.800000
Name: PersonRating, dtype: float64

In [128]:
clean_data

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,placed_at,picked_at,Time_taken (min),Type_of_order,Weather_conditions,Road_traffic_density,Festival,City,Vehicle_condition,Type_of_vehicle,multiple_deliveries
0,0xcdcd,DEHRES17DEL01,36,4.2,2022-02-12 21:55:00,2022-02-12 22:10:00,46,Snack,Fog,Jam,No,Metropolitian,2,motorcycle,3.0
1,0xd987,KOCRES16DEL01,21,4.7,2022-02-13 14:55:00,2022-02-13 15:05:00,23,Meal,Stormy,High,No,Metropolitian,1,motorcycle,1.0
2,0x2784,PUNERES13DEL03,23,4.7,2022-03-04 17:30:00,2022-03-04 17:40:00,21,Drinks,Sandstorms,Medium,No,Metropolitian,1,scooter,1.0
3,0xc8b6,LUDHRES15DEL02,34,4.3,2022-02-13 09:20:00,2022-02-13 09:30:00,20,Buffet,Sandstorms,Low,No,Metropolitian,0,motorcycle,0.0
4,0xdb64,KNPRES14DEL02,24,4.7,2022-02-14 19:50:00,2022-02-14 20:05:00,41,Snack,Fog,Jam,No,Metropolitian,1,scooter,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45578,0x1178,RANCHIRES16DEL01,35,4.2,2022-03-08 21:45:00,2022-03-08 21:55:00,33,Drinks,Windy,Jam,No,Metropolitian,2,motorcycle,1.0
45579,0x7c09,JAPRES04DEL01,30,4.8,2022-03-24 11:35:00,2022-03-24 11:45:00,32,Meal,Windy,High,No,Metropolitian,1,motorcycle,0.0
45580,0xd641,AGRRES16DEL01,21,4.6,2022-02-16 19:55:00,2022-02-16 20:10:00,36,Buffet,Windy,Jam,No,Metropolitian,0,motorcycle,1.0
45582,0x5eee,COIMBRES11DEL01,20,4.7,2022-03-07 13:35:00,2022-03-07 13:40:00,26,Snack,Cloudy,High,No,Metropolitian,0,motorcycle,1.0


In [129]:
final_clean_data = pd.merge(
    left=clean_data,
    right=final_data,
    left_on=['Delivery_person_ID'],
    right_on=['Delivery_person_ID'],
    how='inner'
).drop(['Delivery_person_Age', 'Delivery_person_Ratings'], axis=1)

In [130]:
final_clean_data = final_clean_data[['ID', 'Delivery_person_ID', 'PersonAge', 'PersonRating',
                         'placed_at', 'picked_at', 'Time_taken (min)', 'Type_of_order',
                         'Weather_conditions', 'Road_traffic_density', 'Festival', 'City' ,'Vehicle_condition', 'Type_of_vehicle', 'multiple_deliveries']]

In [131]:
final_clean_data.rename(columns={'Time_taken (min)': 'Time_taken',
                                 'Delivery_person_ID': 'Person_id',
                                 'ID':'UID',
                                 'PersonAge': 'Person_age',
                                 'PersonRating': 'Person_rating'}, inplace=True)

In [132]:
final_clean_data.to_csv("D:/Projects/Zomato Delivery Dataset/clean_zomato_delivery_dataset.csv")

In [133]:
data = final_clean_data
data

,UID,Person_id,Person_age,Person_rating,placed_at,picked_at,Time_taken,Type_of_order,Weather_conditions,Road_traffic_density,Festival,City,Vehicle_condition,Type_of_vehicle,multiple_deliveries
0,0xcdcd,DEHRES17DEL01,29,4.2,2022-02-12 21:55:00,2022-02-12 22:10:00,46,Snack,Fog,Jam,No,Metropolitian,2,motorcycle,3.0
1,0xd987,KOCRES16DEL01,33,4.0,2022-02-13 14:55:00,2022-02-13 15:05:00,23,Meal,Stormy,High,No,Metropolitian,1,motorcycle,1.0
2,0x2784,PUNERES13DEL03,30,2.9,2022-03-04 17:30:00,2022-03-04 17:40:00,21,Drinks,Sandstorms,Medium,No,Metropolitian,1,scooter,1.0
3,0xc8b6,LUDHRES15DEL02,30,4.1,2022-02-13 09:20:00,2022-02-13 09:30:00,20,Buffet,Sandstorms,Low,No,Metropolitian,0,motorcycle,0.0
4,0xdb64,KNPRES14DEL02,30,4.5,2022-02-14 19:50:00,2022-02-14 20:05:00,41,Snack,Fog,Jam,No,Metropolitian,1,scooter,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35227,0x1178,RANCHIRES16DEL01,31,2.5,2022-03-08 21:45:00,2022-03-08 21:55:00,33,Drinks,Windy,Jam,No,Metropolitian,2,motorcycle,1.0
35228,0x7c09,JAPRES04DEL01,28,3.6,2022-03-24 11:35:00,2022-03-24 11:45:00,32,Meal,Windy,High,No,Metropolitian,1,motorcycle,0.0
35229,0xd641,AGRRES16DEL01,31,3.8,2022-02-16 19:55:00,2022-02-16 20:10:00,36,Buffet,Windy,Jam,No,Metropolitian,0,motorcycle,1.0
35230,0x5eee,COIMBRES11DEL01,31,3.5,2022-03-07 13:35:00,2022-03-07 13:40:00,26,Snack,Cloudy,High,No,Metropolitian,0,motorcycle,1.0


# Analyzing Data

### Q. How can we improve the deliveries such that it takes the minimum amount of time to deliver the item in the most efficient way?
### A. For this we need to understand the factors affecting the delivery time, and provide suggestions according to that,

In [134]:
# First factor is the weather conditions
data.groupby('Weather_conditions')['Time_taken'].mean()

Weather_conditions
Cloudy        28.984799
Fog           28.973561
Sandstorms    26.050732
Stormy        26.077557
Sunny         21.978653
Windy         26.301636
Name: Time_taken, dtype: float64

In [135]:
data['weather_status'] = np.select(condlist=[data['Weather_conditions'] == 'Sunny',
                                             data['Weather_conditions'].isin(['Windy', 'Stormy', 'Sandstorms']),
                                             data['Weather_conditions'].isin(['Cloudy', 'Fog'])],
          choicelist=['Good Weather',
                      'Moderate Weather',
                      'Extreme Weather'],
          default='')

In [136]:
weather_agg = data.groupby('weather_status')['Time_taken'].mean()
weather_agg

weather_status
Extreme Weather     28.979123
Good Weather        21.978653
Moderate Weather    26.142994
Name: Time_taken, dtype: float64

In [137]:
(weather_agg - weather_agg.loc['Good Weather'])/weather_agg.loc['Good Weather'] * 100

weather_status
Extreme Weather     31.851224
Good Weather         0.000000
Moderate Weather    18.947211
Name: Time_taken, dtype: float64

### Factor - Weather Conditions
#### Observations :- 
        1. As you can see when the weather is sunny and good, it takes the least amount of time to deliver the item
        2. But if the weather is Moderate, the average delivery time increases by ~ 18%
        3. During Extreme weather conditions, the avg delivery time increases by ~ 31% 
#### Effect :- 
        1. This causes a delay in deliveries due to which orders can get canceled, contributing to a bad User Experience.
        2. Extreme weather conditions increses the risk of safty for the delivery partners
#### Suggestions:-
        1. Whenever the weather conditions are moderate, a message should be displayed to the customers, indicating a slight delay in the delivery due to bad weather,
           This promotes transparency, also a buffer time should be added to the normal expected delivery time so that the customer expects this is the max time for the delivery, it cant get any late than that
           Buffer should be the time contributing to the %age increase in Average delivery time due to bad weather like ~ 5 minutes
        2. During Extreme weathers, for the safety of delivery partners, restrictions should be imposed on placing orders, and should be resumed once the weather clears.

In [138]:
# Second Factor are the Traffic conditions
data.groupby('Road_traffic_density')['Time_taken'].mean()

Road_traffic_density
High      27.281150
Jam       31.192328
Low       21.328266
Medium    26.789558
Name: Time_taken, dtype: float64

In [139]:
data['traffic_status'] = np.select(
    condlist=[data['Road_traffic_density'] == 'Low',
              data['Road_traffic_density'].isin(['Medium', 'High']),
              data['Road_traffic_density'] == 'Jam'],
    choicelist=['None to Low',
                'Moderate to High',
                'Extremely High'],
    default=''
)

In [140]:
traffic_agg = data.groupby('traffic_status')['Time_taken'].mean()
traffic_agg

traffic_status
Extremely High      31.192328
Moderate to High    26.928337
None to Low         21.328266
Name: Time_taken, dtype: float64

In [141]:
percent_increase = ((traffic_agg - traffic_agg.loc['None to Low'])/traffic_agg.loc['None to Low'] * 100).round(1)

In [142]:
# Now we have to understand what time day has the most traffic (Rush Hour)
# For this we are going to break the day into following segments

# Before 12 - Morning
# 12 - 4 - Afternoon
# 4 - 7 - Evening
# 7 - 10 - Night
# After 10 - Late Night

# The column we will choose for this is picked at, because to view the traffic statistics, the item must have left the restaurant.

In [143]:
data['day_segments_for_traffic'] = np.select(
    condlist=[(data['picked_at'].dt.hour < 12) & (data['picked_at'].dt.hour >= 8),
              (data['picked_at'].dt.hour >= 12) & (data['picked_at'].dt.hour < 16),
              (data['picked_at'].dt.hour >= 16) & (data['picked_at'].dt.hour < 19),
              (data['picked_at'].dt.hour >= 19) & (data['picked_at'].dt.hour < 22),
              (data['picked_at'].dt.hour >= 22)],
    choicelist=['Morning',
                'Afternoon',
                'Evening',
                'Night',
                'Late Night'],
    default=''
)

In [144]:
agg = (data.groupby(['traffic_status', 'day_segments_for_traffic'])['UID'].count()/data['UID'].count() * 100).round(1).reset_index().sort_values(['traffic_status', 'UID'], ascending=[True, False]).set_index(['traffic_status', 'day_segments_for_traffic'])


In [145]:
pd.merge(left=percent_increase, right=agg, left_index=True, right_index=True, how='inner').rename(columns={'Time_taken':'%age increase', 'UID': 'percent deliveries'})

%age increase  percent deliveries
traffic_status   day_segments_for_traffic                                   
Extremely High   Night                              46.2                30.7
                 Late Night                         46.2                 1.2
Moderate to High Evening                            26.3                22.1
                 Afternoon                          26.3                 7.6
                 Morning                            26.3                 3.9
                 Night                              26.3                 1.0
None to Low      Late Night                          0.0                19.8
                 Morning                             0.0                13.6

#### Factor - Traffic Conditions

#### Observation :-
        1. When Traffic is Low to none, it takes the least amount of time to deliver the item
        2. When Traffic spikes to a Moderate level, the avg delivery time increses by ~26%
        3. When the Traffic is extremely high leading to a jam, the avg delivery time increses by ~46%
        4. About 30% of the overall deliveries suffer from extreme traffic during night hours
        5. About 22% of the overall deliveries suffer from moderate traffic during evening hours
        
#### Effect :-
        Avg delivery time increses by 46% for about 30% of the overall deliveries because of extreme traffic conditions during night hours
        and by 26% for 22% of the overall deliveries during the evening because of moderate traffic conditions during evening hours, 
        this causes a delay in the deliveries at night and evening,
        due to which there is a high probability of orders/deliveries getting cancelled at night and evening
        ,leading to a bad user experience
        
#### Suggestions :-
        1. Better/alternate routes must be available through navigation systems to the delivery partners to avoid heavy traffic areas, this will help in reducing the time taken to deliver the item
        2. Since extreme to moderate traffic conditions occur majorly during the night and evening hours, a message should be displayed that ordering items during these hours may take longer than expected.
        3. We can increase the delivery charges during the night and evening hours, this will help grow the business economically.
        4. If the time it takes to deliver the items goes beyond a certain threshold, we can implement a strategy to provide free deliveries, this will improve customer satisfaction and loyalty


In [146]:
fest_agg = data.groupby(['Festival'])['Time_taken'].mean()
fest_agg

Festival
No     26.034168
Yes    45.530846
Name: Time_taken, dtype: float64

In [147]:
(fest_agg - fest_agg.loc['No'])/fest_agg.loc['No'] * 100

Festival
No      0.000000
Yes    74.888808
Name: Time_taken, dtype: float64

#### Factor - Festivals
 
#### Observation :-
        1. On normal days, it takes the least amount of time to deliver the items
        2. And during festivals avg delivery time increases by almost 75%

#### Effect :-
        Delay in deliveries promotes to customer unsatisfaction,
        and unsatisfied customers leads to less no of orders and bad business.
        
#### Suggestions :-
        Since Festivals are one of the factors we cant control,
        but we can see it as an opportunity to grow business.
        1. To keep the customers satisfactory despite of late deliveries, we can either provide festival discounts on orders or include a gift
        2. Another strategy to reduce the delay in deliveries can be prioritizing deliveries into HIGH, MEDIUM, STANDARD at additional cost.  

In [148]:
data['D_person_rating_status'] = np.select(
    condlist=[data['Person_rating'] <= 3.5,
              data['Person_rating'] <= 4.5,
              data['Person_rating'] > 4.5],
    choicelist=['Low Rating', 'Medium Rating', 'High Rating'],
    default=''
)

In [149]:
data.groupby('D_person_rating_status')['Time_taken'].mean()

D_person_rating_status
High Rating      25.281720
Low Rating       26.674509
Medium Rating    26.320371
Name: Time_taken, dtype: float64

### Does the ratings of a delivery person affect the time they take to deliver the items. (Correlation Ratings vs Delivery time)

#### Observation :-
        From the statistics above, it does not seem that ratings have huge impact on average delivery time
        but Person with low or medium ratings tends to take more time to deliver the items compared to person with high ratings.

#### Effect :-
        Since ratings are provided by customers, 
        Bad ratings by the customers can be a concern of bad behaviour of the delivery person towards the customer,
        Intentionally late deliveries, poor customer experience etc.

#### Suggestions :-
        1. To motivate delivery persons towards deliveries, we can provide monetary incentives on the basis of their ratings and number of deliveries made each month.
        2. In case of extreme delay in deliveries by a low or medium rating person, strict action must be taken against them (Judging on individual basis) 
        or to improve this proper training and guidance must be provided to them.

In [150]:
# Now let's understand what are the peak hours to place an order, and suggest some strategy for business growth

data['day_segments_for_peak_hours'] = np.select(
    condlist=[(data['placed_at'].dt.hour < 12) & (data['placed_at'].dt.hour >= 8),
              (data['placed_at'].dt.hour >= 12) & (data['placed_at'].dt.hour < 16),
              (data['placed_at'].dt.hour >= 16) & (data['placed_at'].dt.hour < 19),
              (data['placed_at'].dt.hour >= 19) & (data['placed_at'].dt.hour < 22),
              (data['placed_at'].dt.hour >= 22)],
    choicelist=['Morning',
                'Afternoon',
                'Evening',
                'Night',
                'Late Night'],
    default=''
)

In [151]:
# Delivery Distribution
(data.groupby('day_segments_for_peak_hours')['UID'].count()/data['UID'].count() * 100).round(1)

day_segments_for_peak_hours
Afternoon      7.3
Evening       22.9
Late Night    19.8
Morning       18.1
Night         32.0
Name: UID, dtype: float64

### Understanding the peak hours to place an order, and suggest some strategy for business growth
    [Note - Since each delivery corresponds to at-least 1 order that is being delivered
     So this delivery distribution is directly proportional to the Order distribution.]

#### Observation :- 
        At Night time, Max no. of Orders are placed, i.e. Peak hours for business are between 7 PM to 10 PM
        And Moderate business hours are in the evening (4 PM - 7PM), Late Night (After 10 PM) and in the Morning (8 AM - 12 PM)

#### Effect :-
        During peak hours, Max no. of Orders are getting placed, which promotes business growth
        And, the efforts to prepare the Orders also increases.

#### Suggestions :-
        - To gain benefits from the peak hours economically, we can increase the cost of the orders,
          i.e. the greater the demand the higher the price
        - To handle workload during peak hours, ensure proper logistics for the deliveries, this includes hiring more delivery partners
        - We can also implement some marketing/promotional strategy during normal hours to increase the business at that time, this can help in evenly distributing order placements
        
        

In [152]:
city_del_agg = (data.groupby('City')['UID'].count()/data['UID'].count() * 100).round(1)

In [153]:
city_time_agg = data.groupby('City')['Time_taken'].mean().round(1)

In [154]:
pd.merge(left=city_del_agg, right=city_time_agg, left_index=True, right_index=True, how='inner')

,UID,Time_taken
City,,
Metropolitian,74.9,27.4
Semi-Urban,0.4,49.7
Urban,24.7,23.0


In [155]:
data.groupby('multiple_deliveries')['Time_taken'].mean()
# The below data shows that for
# No multiple deliveries, it takes the least amt of time to deliver the items (on average basis)
# and for 3 or more deliveries, avg delivery time increases by ~75% and goes beyond ~100%

multiple_deliveries
0.0    23.011716
1.0    26.954760
2.0    40.366645
3.0    47.867857
Name: Time_taken, dtype: float64

### Distribution of deliveries based on different city locations 
### and Avg Delivery time for each different city locations

#### Observation :-
    About 74.9% of the deliveries are from the metropolitan areas
    24.7% of the deliveries are from the Urban Areas, for which it takes the least amount of time to deliver the items
    Almost no deliveries are from the Semi-Urban Areas, for which it takes the largest amount of time to deliver the items

#### Effects :-
    This suggest that maximum no. of customers are from metropolitan areas, 
    for which we have a fairly strong delivery network because avg delivery time is only 4.4 min (~19%) greater than the minimum avg delivery time
    Also, Semi-Urban areas has the least no. of customers which can be due to unfeasible delivery location in terms of delivering the items within the expected timeframe

#### Suggestions :-
    Try to fortify the delivery network more in the metropolitan and urban areas compared to semi-Urban,
    Because if the profit from that delivery is getting converted into a loss because of the distance, then it's a waste of resources and efforts
    To secure the delivery network we can hire more delivery staff, that would deliver multiple items at the same time reducing delivery delays (as seen from the data above)

In [156]:
data.groupby('Type_of_vehicle')['Time_taken'].mean()

Type_of_vehicle
electric_scooter    24.539571
motorcycle          27.708970
scooter             24.602850
Name: Time_taken, dtype: float64

In [157]:
# Distribution of deliveries for each type of vehicle
data.groupby('Type_of_vehicle')['UID'].count()/(data['UID'].count()) * 100

Type_of_vehicle
electric_scooter     8.069369
motorcycle          58.662579
scooter             33.268052
Name: UID, dtype: float64

In [158]:
# Effect of vehicle condition on deliveries
data['Vehicle_cond_status'] = np.select(
    condlist=[data['Vehicle_condition'] == 0, data['Vehicle_condition'] == 1],
    choicelist=['Bad Condition', 'Moderate Condition'],
    default='Good Condition'
)
data.groupby('Vehicle_cond_status')['Time_taken'].mean()

Vehicle_cond_status
Bad Condition         30.176231
Good Condition        24.571903
Moderate Condition    24.485800
Name: Time_taken, dtype: float64

### Effect of vehicle and its condition on deliveries

#### Observations :-
        1. Motorcycle takes the longest amount of time to deliver the items compared to scooter and electric_scooter
        2. Almost 58% of the deliveries happen through motorcycles
        3. Vehicles with bad condition take the longest amount of time to deliver the items compared to Good and Moderate condition vehicles.

#### Effects :-
        Bad condition vehicles not only put the delivery partner at risk but can also cause unexpected delay in deliveries.
        Motorcycles not only causes a delay in deliveries but also limits the storage capacity for the order.
        
#### Suggestions :-
        Since Scooter and electric_scooters takes the least amount of time to deliver the item, they should replace Motorcycles in that regard to reduce the avg delivery time.
        Proper check must be performed on delivery vehicles at least twice a month, to ensure safety and reduced delivery time.

In [159]:
# Some KPIs

In [160]:
# Average Delivery Time
Overall_avg_delivery_time = data['Time_taken'].mean()
Overall_avg_delivery_time

np.float64(26.419873978201636)

In [169]:
# Average delivery rate for each month

In [168]:
data.groupby(data['picked_at'].dt.month)['UID'].count()

picked_at
2     5587
3    24760
4     4885
Name: UID, dtype: int64

In [167]:
data.groupby(data['picked_at'].dt.month)['UID'].count().sum()/12

np.float64(2936.0)